# Task 5 - Learning crowd dynamics

In [11]:
%load_ext autoreload
%autoreload 2

from helpers.data import load_mi_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
load_mi_data('data/MI_timesteps.txt', skip_first=1000)

,areaDensityCounting-PID5,areaDensityCounting-PID6,areaDensityCounting-PID7,areaDensityCounting-PID8,areaDensityCounting-PID9,areaDensityCounting-PID10,areaDensityCounting-PID11,areaDensityCounting-PID12,areaDensityCounting-PID13
timeStep,,,,,,,,,
1001,0,2,97,7,0,0,5,1,43
1002,0,2,96,7,0,0,5,1,45
1003,0,2,95,6,0,0,5,0,46
1004,0,2,94,6,0,0,4,1,46
1005,0,2,92,6,0,0,5,0,46
...,...,...,...,...,...,...,...,...,...
14997,0,10,95,4,0,0,2,1,39
14998,0,10,94,3,0,0,3,1,39
14999,0,10,93,3,0,1,2,1,40
